## Load data

In [ ]:
from collections import Counter
import pandas as pd
import numpy as np
import os
import re

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
remove_these = set(stopwords.words('english'))

In [ ]:
REMOVE_STOPWORDS = True

In [ ]:
%%time

with open('../input/musicxmatch-dataset/mxm_dataset_train.txt','r') as f:
    lines = f.readlines()
    words = lines[17].replace('%','').split(',')
    all_songs_dict = dict()
    for i,l in list(enumerate(lines))[18:]:
        song_info = l.split(',')
        MSDID = song_info[0]
        song_bow = [x.split(':') for x in song_info[2:]]
        song_dict = {}
        for word, word_count in song_bow:
            song_dict[int(word)] = int(word_count.replace('\n',''))
        word_lists = [[words[word-1]]*song_dict[word] for word in song_dict.keys()]
        song = [word for word_list in word_lists for word in word_list]
        if REMOVE_STOPWORDS:
            song = [w for w in song if w not in remove_these]
        all_songs_dict[str(MSDID)] = ' '.join(song).replace('\n','')

Data:
1. song_info
2. all_songs_dict

In [ ]:
print(len(all_songs_dict.keys()))
song_msd_ids = list(all_songs_dict.keys())

In [ ]:
print(np.random.choice(song_msd_ids))

In [ ]:
%%time

all_song_meta_dict = dict()
with open('../input/musicxmatch-779k-matches/mxm_779k_matches.txt','r') as f:
    lines = f.readlines()
    for i in range(18, len(lines)):
        line = lines[i].split('<SEP>')
        MSDID = line[0]
        artist = line[1]
        title = line[2]
        all_song_meta_dict[str(MSDID)] = {'artist': artist, 'title': title}

In [ ]:
len(all_song_meta_dict.keys())

In [ ]:
# all_artists = sorted(set([all_song_meta_dict[msdId]['artist'].lower() for msdId in all_song_meta_dict]))

In [ ]:
d = {
    'MSDID': list(all_songs_dict.keys()),
    'cleaned_text': [all_songs_dict[x] for x in all_songs_dict.keys()]
    }
msdid_df = pd.DataFrame.from_dict(d)
print(msdid_df.shape)
msdid_df.head()

In [ ]:
d = {
    'MSDID': msdid_df['MSDID'],
    'artist': [all_song_meta_dict[x]['artist'] for x in msdid_df['MSDID']],
    'title': [all_song_meta_dict[x]['title'] for x in msdid_df['MSDID']]
    }
meta_df = pd.DataFrame.from_dict(d)
print(meta_df.shape)
meta_df.head()

In [ ]:
full_df_ = pd.merge(msdid_df, meta_df, on='MSDID', how='left')
print(full_df_.shape)
full_df_.head()

In [ ]:
full_df_.loc[(full_df_.artist == 'Nirvana') & (full_df_.title == 'Dumb')].cleaned_text.values

In [ ]:
my_artists_ = ['Green Day','Blink-182','Sum 41','Rise Against',
              'Four Year Strong','Three Days Grace',
              'Tame Impala','Snoop Dogg','Thin Lizzy','The Strokes',
              'Nirvana','All Time Low', 'Foo Fighters','Lil Wayne',
              'Eminem','Tom Petty','Seether', 'Shinedown',
              'Red Hot Chili Peppers','The Who', 'The Doors',
              'Linkin Park','Arctic Monkeys','The Black Keys',
              'Led Zeppelin','The Kinks','Collective Soul',
              'Avenged Sevenfold','George Harrison','Eric Clapton',
              'The Donnas','System of a Down', 'MxPx',
              'Weezer','Rancid','Atmosphere','Santigold',
              'Iron Maiden','Fall Out Boy', 'Hank Williams']

my_artists = ['The Smashing Pumpkins', 'Bruce Springsteen','Beastie Boys']

sampled_artists = list(np.random.choice(full_df_['artist'], size=500, replace=False))
# sampled_artists

In [ ]:
# top_artists = full_df_['artist'].value_counts()[:10].reset_index()['index'].values
# full_df = full_df_[full_df_['artist'].isin(my_artists)].reset_index(drop=True)
# full_df = full_df_[full_df_['artist'].isin(sampled_artists)].reset_index(drop=True)
# full_df = full_df_[full_df_['artist'].isin(sampled_artists + my_artists + my_artists_)].reset_index(drop=True)
full_df = full_df_.copy()
print(full_df.shape)
full_df.head()

## Train model and get embeddings

In [ ]:
data = full_df['cleaned_text'].tolist()

# Not used

In [ ]:
%%time

# get training data
data = full_df['cleaned_text'].tolist()
# test_data = test_df['cleaned_text'].tolist()
#train_corpus = [TaggedDocument(words=word_tokenize(dat.lower()), tags=[str(i)]) for i, dat in list(enumerate(data))]
# build model
# model = Doc2Vec(vector_size=50, min_count=1, epochs=10, dm=0)
# model.build_vocab(train_corpus)
# # train model
# model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
#model.save('trained_model_example')

In [ ]:
file = '../input/d2v-model-for-musicxmatch/trained_model_example'
model = Doc2Vec.load(file)

In [ ]:
emb_df = pd.DataFrame([model.dv[f'{i}'] for i in range(len(full_df))])
fe_df = full_df.copy()
for c in emb_df.columns:
    fe_df[c] = emb_df[c]
# fe_df[fe_df['artist'].isin(my_artists)].groupby('artist').mean().reset_index()

In [ ]:
fe_df.head()

## Visualization with UMAP

In [ ]:
from sklearn.preprocessing import StandardScaler
#from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
import umap

In [ ]:
# %%time
# n = 3

# pipeline = Pipeline([('scaling', StandardScaler()), ('umap', umap.UMAP(n_components=n, random_state=42))])
# song_embedding = pipeline.fit_transform(emb_df)
# # tsne = TSNE(n_components=n, perplexity=30, random_state=42)
# # song_embedding = tsne.fit_transform(song_df)
# umap_df = pd.DataFrame(data = song_embedding, columns = list(range(n)))
# umap_df['title'] = full_df['title']
# umap_df['artist'] = full_df['artist']

In [ ]:
#umap_df.to_csv('umap_3_components.csv', index=False)

In [ ]:
umap_df = pd.read_csv('../input/d2v-model-for-musicxmatch/umap_3_components.csv')
umap_df

In [ ]:
# song_df = ?

In [ ]:
my_artists = ['Blink-182',
              'Tame Impala','Snoop Dogg','The Strokes',
              'Eminem',
              'Red Hot Chili Peppers','The Who',
              'Linkin Park',
              'Avenged Sevenfold','Eric Clapton']
# my_artists = tsne_df['artist'].sample(100)
# my_artists = ['Avenged Sevenfold', 'Eminem','The Strokes']
# my_artists = ['Avenged Sevenfold', 'Bob Dylan', 'Hank Williams','Dolly Parton']
my_artists

In [ ]:
umap_df.artist.value_counts()[:30]

In [ ]:
plot_df = fe_df[fe_df['artist'].isin(my_artists)].reset_index(drop=True)

In [ ]:
emb_df_ = emb_df[fe_df['artist'].isin(my_artists)]

In [ ]:
print(umap_df.shape)
umap_df.head()

In [ ]:
# tsne_df = tsne_df.groupby('artist').mean().reset_index()

In [ ]:
umap_df_ = umap_df[fe_df['artist'].isin(my_artists)].reset_index(drop=True)
umap_df_['title'] = plot_df['title']
umap_df_['artist'] = plot_df['artist']
umap_df_.columns = ['umap dimension 0','umap dimension 1','umap dimension 2','title','artist']
print(umap_df_.shape)
umap_df_.head()

In [ ]:
umap_df_

In [ ]:
import plotly.express as px

for x in [('umap dimension 0','umap dimension 1'),
          ('umap dimension 0','umap dimension 2'),
          ('umap dimension 1','umap dimension 2')]:
    fig = px.scatter(umap_df_, 
                     x=x[0],
                     y=x[1],
                     hover_data=['artist','title'],
                     color='artist',
                     opacity=.8,
                     template='simple_white')\
            .update_traces(marker=dict(size=15))\
            .update_layout(
                title={
                    'text': "Song Lyric Embeddings",
                    'y':0.95,
                    'x':0.19,
                    'xanchor': 'center',
                    'yanchor': 'top'})
    fig.show()

In [ ]:
for x in [('umap dimension 0','umap dimension 1','umap dimension 2')]:   
    fig = px.scatter_3d(umap_df_, 
                     x=x[0],
                     y=x[1],
                     z=x[2],
                     hover_data=['artist', 'title'],
                     color='artist',
                     template='simple_white')
    fig.show()

## BERT approach

In [ ]:
from tqdm import tqdm
from transformers import DistilBertModel, DistilBertTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from torch.utils.data import DataLoader, Dataset
class DATA(Dataset):
    
    def __init__(self, df, max_len):
        super().__init__()
        self.df = df
        self.max_len = max_len
        self.tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        
        inputs = self.tokenizer.encode_plus(
            row,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = torch.LongTensor(inputs['input_ids'])
        mask = torch.LongTensor(inputs['attention_mask'])

        return ids, mask

    def __len__(self):
        return self.df.shape[0]

In [ ]:
types = [type(full_df.cleaned_text.iloc[i]) for i in range(full_df.shape[0])] # every row is a string
all([t == str for t in types]) 

In [ ]:
train_df = DATA(full_df.cleaned_text.apply(str), 50)
train_df[0]

In [ ]:
if torch.cuda.is_available():  # checking availability of the GPU
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

In [ ]:
import sys

def convert_if_not_tensor(x, dtype):
        if isinstance(x, torch.Tensor):
            return x.to(device, dtype=dtype)
        else:
            return torch.tensor(x, dtype=dtype, device=device)

def read_data(data):
    return tuple(d.cuda() for d in data)
def validate(val_loader):
    tbar = tqdm(val_loader, file=sys.stdout)
    embs = []
    model =  DistilBertModel.from_pretrained('distilbert-base-uncased')
    model.cuda()
    model.eval()
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            ids = convert_if_not_tensor(data[0], dtype=torch.long)
            mask = convert_if_not_tensor(data[1], dtype=torch.long)
            inputs = read_data(data)

            preds = model(inputs[0], inputs[1])[0]
            preds = preds[:, 0, :]
            embs.append(preds.detach().cpu().numpy())
    del preds, inputs
    torch.cuda.empty_cache()
    return np.concatenate(embs)

In [ ]:
BS = 16
NW = 2
data_loader = DataLoader(train_df, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)

In [ ]:
embs_ = validate(data_loader)

In [ ]:
import pickle
with open('embeddings_distilbert.pickle', 'wb') as file:
    pickle.dump(embs_, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
embs_[0].shape

In [ ]:
emb_df = pd.DataFrame([embs_[i] for i in range(len(full_df))])
fe_df = full_df.copy()
for c in emb_df.columns:
    fe_df[c] = emb_df[c]

In [ ]:
%%time
n = 3

pipeline = Pipeline([('scaling', StandardScaler()), ('umap', umap.UMAP(n_components=n, random_state=42))])
song_embedding = pipeline.fit_transform(emb_df)
umap_df = pd.DataFrame(data = song_embedding, columns = list(range(n)))
umap_df['title'] = full_df['title']
umap_df['artist'] = full_df['artist']

In [ ]:
umap_df.to_csv('umap_3_components_bert.csv', index=False)
# umap_df = pd.read_csv('umap_3_components_bert.csv')

In [ ]:
plot_df = fe_df[fe_df['artist'].isin(my_artists)].reset_index(drop=True)
emb_df_ = emb_df[fe_df['artist'].isin(my_artists)]

In [ ]:
umap_df_ = umap_df[fe_df['artist'].isin(my_artists)].reset_index(drop=True)
umap_df_['title'] = plot_df['title']
umap_df_['artist'] = plot_df['artist']
umap_df_.columns = ['umap dimension 0','umap dimension 1','umap dimension 2','title','artist']
print(umap_df_.shape)
umap_df_.head()

In [ ]:
import plotly.express as px

for x in [('umap dimension 0','umap dimension 1'),
          ('umap dimension 0','umap dimension 2'),
          ('umap dimension 1','umap dimension 2')]:
    fig = px.scatter(umap_df_, 
                     x=x[0],
                     y=x[1],
                     hover_data=['artist','title'],
                     color='artist',
                     opacity=.8,
                     template='simple_white')\
            .update_traces(marker=dict(size=15))\
            .update_layout(
                title={
                    'text': "Song Lyric Embeddings",
                    'y':0.95,
                    'x':0.19,
                    'xanchor': 'center',
                    'yanchor': 'top'})
    fig.show()

In [ ]:
for x in [('umap dimension 0','umap dimension 1','umap dimension 2')]:   
    fig = px.scatter_3d(umap_df_, 
                     x=x[0],
                     y=x[1],
                     z=x[2],
                     hover_data=['artist', 'title'],
                     color='artist',
                     template='simple_white')
    fig.show()